In [1]:
import tensorflow as tf
import numpy
import os
import json, pickle
import pandas
from functools import partial, reduce
import importlib

import sys
sys.path.append('../libs')

import flacdb
import prepare_data
import initialize
import data_pipeline
import conv_model
import plot_batch
import generate_report_pt
import load_diagnosis

In [2]:
!nvidia-smi

Mon May 11 13:15:07 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.87.00    Driver Version: 418.87.00    CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX TIT...  On   | 00000000:05:00.0 Off |                  N/A |
| 22%   32C    P8    16W / 250W |  11853MiB / 12212MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [2]:
# initialize.partition_subject_ids()

In [ ]:
%%time

%matplotlib widget

# model_id = 1458429
model_id = 1464849
plotter, model, X, Y, P = generate_report_pt.run(
    model_id = model_id,
    checkpoint_index = -1,
    example_count_log2 = 14,
    path = '/scr1/mimic/initial_data_{}/'.format(model_id),
#     path = '/scr1/mimic/initial_data/',
    dark = True
)
plotter()

found hypes ../hypes/1464849_20200509-214014.json 
found weights /scr1/checkpoints/1464849_20200509-214014_00384.ckpt


In [3]:
P.shape

(13971, 96)

In [4]:
# mv /scr-ssd/mimic/test_subject_ids.txt /scr-ssd/mimic/test_subject_ids_old.txt

In [5]:
model_id = 1459283
checkpoint_index = 1
example_count_log2 = 14
path = '/scr1/mimic/initial_data/'

ckpts = os.listdir('/scr1/checkpoints')
ckpts = sorted(i for i in ckpts if 'index' in i and str(model_id) in i)
hypes_path = '../hypes/{}.json'.format(ckpts[0].split('.')[0][:-6])
weights_path = '/scr1/checkpoints/' + ckpts[checkpoint_index]
assert(os.path.isfile(hypes_path) and os.path.isfile(weights_path))
weights_path = weights_path.replace('.index', '')
print('found hypes', hypes_path, '\nfound weights', weights_path)

H0 = json.load(open(hypes_path))
H = initialize.load_hypes()
H = {**H, **H0}
H['batch_size_validation_log2'] = 7

part = 'validation'
tensors, metadata, priors = initialize.run(H, parts=[part], load_path=path)
dataset = data_pipeline.build(H, tensors[part], part)

found hypes ../hypes/1459283_20200505-214200.json 
found weights /scr1/checkpoints/1459283_20200505-214200_00256.ckpt


In [8]:
x, y = next(iter(dataset))

In [20]:
(x['signals'].numpy().sum(axis=1) == 0).mean()

0.6

In [26]:
tensors['validation']['sig_index'][0]

<tf.RaggedTensor [[[2, 1, 6, 0, 0, 0, 0, 3, 0, 4, 5, 0, 0, 0, 0]]]>

In [2]:
model_id = 1459283
checkpoint_index = 1
example_count_log2 = 14
path = '/scr1/mimic/initial_data/'

ckpts = os.listdir('/scr1/checkpoints')
ckpts = sorted(i for i in ckpts if 'index' in i and str(model_id) in i)
hypes_path = '../hypes/{}.json'.format(ckpts[0].split('.')[0][:-6])
weights_path = '/scr1/checkpoints/' + ckpts[checkpoint_index]
assert(os.path.isfile(hypes_path) and os.path.isfile(weights_path))
weights_path = weights_path.replace('.index', '')
print('found hypes', hypes_path, '\nfound weights', weights_path)

H0 = json.load(open(hypes_path))
H = initialize.load_hypes()
H = {**H, **H0}
H['batch_size_validation_log2'] = 7

part = 'validation'
tensors, metadata, priors = initialize.run(H, parts=[part], load_path=path)
dataset = data_pipeline.build(H, tensors[part], part)

batch_count = 2 ** (example_count_log2 - H['batch_size_validation_log2'])
model = conv_model.build(H, priors)
model.load_weights(weights_path)
X, Y, P = generate_report_pt.get_predictions(H, model, dataset, weights_path, batch_count)

y_true, y_pred = Y['diagnosis'], P
sigs = ['PLETH', 'II', 'V', 'AVR', 'RESP', 'ABP']
sig_index = [H['input_sigs'].index(i) for i in sigs]
x = X['signals'][:, :, sig_index]
M = metadata.reset_index()[['subject_id', 'rec_id']].drop_duplicates()
M = M.set_index('rec_id', verify_integrity=True)
subject_ids = M.loc[Y['rec_id']].values[:, 0]
codes = priors.index.to_list()

unique_ids = sorted(set(subject_ids))
y_true_ = [y_true[subject_ids == i].max(axis=0) for i in unique_ids]
y_true_ = numpy.stack(y_true_)
y_pred_ = [y_pred[subject_ids == i].max(axis=0) for i in unique_ids]
#     y_pred_ = [y_pred[subject_ids == i].mean(axis=0) for i in unique_ids]
y_pred_ = numpy.stack(y_pred_)
# x_ = numpy.stack([
#     x[subject_ids == i][y_pred[subject_ids == i].max(axis=1).argmax()] 
#     for i in unique_ids
# ])

found hypes ../hypes/1459283_20200505-214200.json 
found weights /scr1/checkpoints/1459283_20200505-214200_00256.ckpt
loading predictions


In [4]:
i = unique_ids[0]
I = subject_ids == i
[y_pred[I, j].argmax() for j in range(y_pred.shape[1])]

[7,
 3,
 1,
 1,
 10,
 10,
 6,
 1,
 7,
 8,
 6,
 6,
 10,
 7,
 0,
 0,
 3,
 10,
 0,
 3,
 10,
 6,
 6,
 3,
 6,
 6,
 6,
 8,
 8,
 5,
 8,
 8,
 5,
 5,
 6,
 5,
 9,
 8,
 9,
 6,
 8,
 9,
 6,
 6,
 6,
 6,
 10,
 6,
 7,
 5,
 5,
 3,
 10,
 6,
 1,
 3,
 6,
 6,
 3,
 10,
 7,
 7,
 7,
 7,
 7,
 7,
 7,
 10,
 10,
 10,
 7,
 6,
 6,
 6,
 6,
 6,
 7,
 0,
 7,
 7,
 10,
 10,
 5,
 3,
 10,
 6,
 3,
 6,
 6,
 3]

In [7]:
y_pred[I].argmax(axis=0)

array([ 7,  3,  1,  1, 10, 10,  6,  1,  7,  8,  6,  6, 10,  7,  0,  0,  3,
       10,  0,  3, 10,  6,  6,  3,  6,  6,  6,  8,  8,  5,  8,  8,  5,  5,
        6,  5,  9,  8,  9,  6,  8,  9,  6,  6,  6,  6, 10,  6,  7,  5,  5,
        3, 10,  6,  1,  3,  6,  6,  3, 10,  7,  7,  7,  7,  7,  7,  7, 10,
       10, 10,  7,  6,  6,  6,  6,  6,  7,  0,  7,  7, 10, 10,  5,  3, 10,
        6,  3,  6,  6,  3])

In [8]:
x_ = []
for i in unique_ids:
    I = subject_ids == i
    x_.append(x[I][y_pred[I].argmax(axis=0)])

In [ ]:
x_ = numpy.stack([
    x[subject_ids == i][y_pred[subject_ids == i].argmax(axis=0)]
    for i in unique_ids
])

In [9]:
numpy.stack(x_).shape

(885, 90, 2048, 6)

In [6]:
y_pred.shape

(14778, 90)

In [7]:
x_ = numpy.stack([
    x[subject_ids == i][y_pred[subject_ids == i].max(axis=1).argmax()] 
    for i in unique_ids
])
x_.shape

(885, 2048, 6)

In [10]:
i, j = unique_ids[0], 5
y_pred[subject_ids == i, j].argmax()

10

In [5]:
x.shape

(14778, 2048, 6)

In [ ]:
x_ = numpy.stack([
    [
        x[subject_ids == i][y_pred[subject_ids == i, j].argmax()]
        for j in range(y_pred.shape[1])
    ]
    for i in unique_ids
])
x_.shape

In [2]:
parts = ['train', 'validation']
path = '/scr1/mimic/initial_data/'
H = initialize.load_hypes()
tensors, metadata, priors = initialize.run(H, parts=parts, load_path=path)
rec_ids = tensors['train']['rec_id']
rec_ids = [k for i in rec_ids.to_list() for j in i for k in j]

NameError: name 'M' is not defined

In [3]:
M = metadata.reset_index()[['subject_id', 'rec_id']].drop_duplicates()
M = M.set_index('rec_id', verify_integrity=True)
M.loc[rec_ids]['subject_id'].unique().shape

(9145,)

In [4]:
rec_ids = tensors['validation']['rec_id']
rec_ids = [k for i in rec_ids.to_list() for j in i for k in j]
M.loc[rec_ids]['subject_id'].unique().shape

(935,)

In [24]:
metadata = pandas.read_hdf('/scr-ssd/mimic/metadata.hdf')
metadata = metadata.reindex(metadata.index & prepare_data.get_serialized())
metadata = metadata[metadata['sig_len'] > prepare_data.CHUNK_SKIP_SIZE]

In [38]:
metadata = pandas.read_pickle(path + 'metadata.pkl')
len(metadata.reset_index()['subject_id'].unique())

10080

In [30]:
diagnosis = pandas.read_pickle(path + 'diagnosis.pkl')

In [32]:
diagnosis.shape

(48894, 90)

In [35]:
diagnosis['428'].sum()

13608

In [36]:
13608 / 48894

0.2783163578353172

In [10]:
# model_id = 1449529
# path = '/scr1/mimic/initial_data_{}/'.format(model_id)
# sig_data, metadata, diagnosis = initialize.load_initial_data(load_path=path)
# diagnosis = load_diagnosis.augment(diagnosis, metadata)
# diagnosis = load_diagnosis.fix(diagnosis)
# priors = (diagnosis == 1).sum() / (diagnosis != 0).sum()
# diagnosis = load_diagnosis.conform(diagnosis, metadata)
H = initialize.load_hypes()
partition = initialize.get_partition(["PLETH", "II", "V", "AVR", "RESP", "ABP"], sig_data)

In [67]:
import importlib

In [18]:
importlib.reload(initialize)

<module 'initialize' from '../libs/initialize.py'>

In [19]:
partition = initialize.get_partition(H['input_sigs_validation'], sig_data)

In [13]:
has_validation_sigs = (sig_data['sig_index'][H['input_sigs_validation']] > 0).all(axis=1)

In [16]:
has_validation_sigs.reset_index()['subject_id'].unique().shape

(10080,)

In [27]:
has_validation_sigs = (sig_data['sig_index'][["PLETH", "II", "V", "AVR", "RESP"]] > 0).all(axis=1)
has_validation_sigs[has_validation_sigs].reset_index()['subject_id'].unique().shape

(4138,)

In [25]:
has_validation_sigs = (sig_data['sig_index'][H['input_sigs_validation']] > 0).all(axis=1)
has_validation_sigs[has_validation_sigs].reset_index()['subject_id'].unique().shape

(1834,)

In [22]:
has_validation_sigs[partition['validation']].reset_index()['subject_id'].unique().shape

(929,)

In [41]:
diagnosis.shape

(284578, 96)

In [20]:
partition['train'].sum()

225248

In [21]:
partition['validation'].sum()

21528

In [53]:
subject_ids = sig_data.index.to_frame()['subject_id']

In [56]:
unique_ids = subject_ids.unique()

In [66]:
numpy.random.permutation(unique_ids)[:len(unique_ids)//5].shape

(2016,)

In [61]:
unique_ids[numpy.random.permutation(len(unique_ids))]

array([15922, 48640, 96732, ..., 25662,  9575, 32229])

In [50]:
sig_data.reset_index()['subject_id'].unique().shape

(10080,)

In [47]:
sig_data.shape

(284578, 117)

In [46]:
7912 / (263413 + 7912)

0.02916060075555146

In [5]:
codes.index('428')

42

In [5]:
len(subject_ids)

13971

In [6]:
unique_ids = sorted(set(subject_ids))

In [7]:
y_true_ = [y_true[subject_ids == i].mean(axis=0) for i in unique_ids]
y_true_ = numpy.stack(y_true_)
y_true_.shape

(383, 96)

In [8]:
y_true_ = [y_true[subject_ids == i] for i in unique_ids]
{tuple(sorted(set(i[:, j]))) for j in range(y_true.shape[1]) for i in y_true_}

{(-1,), (0,), (0, 1), (1,)}

In [10]:
{i.shape[0] for i in y_true_}

{3,
 4,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56}

In [11]:
y_true_ = [y_true[subject_ids == i].max(axis=0) for i in unique_ids]
y_true_ = numpy.stack(y_true_)
y_true_.shape

(383, 96)

In [12]:
y_pred_ = [y_pred[subject_ids == i].max(axis=0) for i in unique_ids]
y_pred_ = numpy.stack(y_pred_)
y_pred_.shape

(383, 96)

In [15]:
(y_true_[:, codes.index('428')] == 1).sum()

107

In [41]:
metadata.reset_index()['subject_id'].unique().shape

(10080,)

In [43]:
metadata['hadm_id'].unique().shape

(10246,)

In [16]:
x.shape

(13971, 2048, 6)

In [22]:
y_pred[subject_ids == unique_ids[0]].argmax()

506

In [24]:
x_ = [x[subject_ids == i][y_pred[subject_ids == i].max(axis=1).argmax()] for i in unique_ids]

In [25]:
x_ = numpy.stack(x_)
x_.shape

(383, 2048, 6)